In [5]:
import os
from pandas import *
from sklearn.mixture import VBGMM

In [8]:
df = read_csv('../boots_dataset/features_women.csv', index_col = 'asin')

In [9]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
asin,,,,,,,,,,,,,,,,,,,,,
B005IYS5ZA,0.0,0.8051,0.0000,0.0,0.000,0.1,0.0000,0.3484,0.0000,1.3334,...,0.0000,0.0,0.0,0.0,0.0000,3.2731,0.0,0.0507,0.0,0.0000
B007KHXTXQ,0.0,0.0000,2.9935,0.0,1.199,0.0,0.0000,0.0000,2.8597,0.0000,...,0.0000,0.0,0.0,0.0,2.6146,0.0000,0.0,0.0201,0.0,0.0000
B00IA6VTO2,0.0,3.2722,0.0000,0.0,0.000,0.0,0.3835,0.0000,0.0000,0.0000,...,3.5435,0.0,0.0,0.0,0.0000,0.7942,0.0,0.0000,0.0,0.0000
B0091UELGO,0.0,0.0000,0.0000,0.0,0.000,0.0,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0,0.0,0.0,0.0000,0.0753,0.0,0.0000,0.0,2.6916
B006CB31NO,0.0,0.0000,0.0000,0.0,0.000,0.0,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0,0.0,0.0,0.0000,1.2655,0.0,0.0000,0.0,3.5446


In [24]:
model = VBGMM(n_components=25, covariance_type='diag', alpha=1.0, random_state=None, 
      thresh=None, tol=0.001, verbose=0, min_covar=None, n_iter=10, 
      params='wmc', init_params='wmc')

In [25]:
nonzero = np.nonzero([0, 1, 1, 0])[0]

In [36]:
subset_cluster = df.iloc[:, nonzero]

In [37]:
model.fit(subset_cluster)

VBGMM(alpha=0.04, covariance_type='diag', init_params='wmc', min_covar=None,
   n_components=25, n_iter=10, params='wmc', random_state=None,
   thresh=None, tol=0.001, verbose=0)

In [38]:
DataFrame(model.predict(subset_cluster)).loc[:, 0].value_counts()

24    2826
16     247
4      187
22     185
7      143
23      64
18      17
3       13
0       12
10       2
Name: 0, dtype: int64

In [47]:
k = 25

cluster_dict = {}
pred = model.predict(subset_cluster)
df.loc[:, 'cluster'] = pred

for i, cluster in enumerate(np.unique(pred)):
    cluster_dict[i] = list(df[df.cluster == cluster].drop('cluster', axis = 1).index)

In [70]:
from pandas import *
from sklearn.mixture import VBGMM
from sklearn.mixture import DPGMM
from sklearn.cluster import KMeans
import sys

class ClusteringImages:
    def __init__(self, df, coef, model_name, alpha, k):
        self.generate_model(model_name, alpha, k)
        self.features_df = df.iloc[:, np.nonzero(coef)[0]]
        self.fit_model()
        self.cluster()
        
    def generate_model(self, model_name, alpha, k):
        if model_name == 'VBGMM':
            self.model = VBGMM(n_components=k, covariance_type='diag', alpha=alpha, random_state=None, 
              thresh=None, tol=0.001, verbose=0, min_covar=None, n_iter=10, 
              params='wmc', init_params='wmc')
        elif model_name == 'DPGMM':
            self.model = DPGMM(n_components=k, covariance_type='diag', alpha=alpha, random_state=None, 
                  thresh=None, tol=0.001, verbose=0, min_covar=None, n_iter=10, 
                  params='wmc', init_params='wmc')        
        elif model_name == 'K-Means':
            self.model = KMeans(n_clusters=k, init='k-means++', n_init=10, max_iter=300, 
                                tol=0.0001, precompute_distances='auto', verbose=0, random_state=None, 
                                copy_x=True, n_jobs=1)
        else:
            print 'enter valid cluster'
            sys.exit(0)
        
    def fit_model(self):
        self.model.fit(self.features_df)
    
    def cluster(self):
        self.cluster_dict = {}
        pred = self.model.predict(self.features_df)
        self.features_df.loc[:, 'cluster'] = pred

        for i, cluster in enumerate(np.unique(pred)):
            self.cluster_dict[i] = list(self.features_df[self.features_df.loc[:, 'cluster'] == cluster]\
                                   .drop('cluster', axis = 1)\
                                   .index)

In [71]:
c = ClusteringImages(df, [0, 0, 1, 2, 3, 0, 0, 1], 'VBGMM', 1, 25)